In [12]:
#I am providing the links for output videos check them out after viewing the code:
#the screenshots of outputs are uploaded in readme.
# https://drive.google.com/file/d/19kMRdTxdOkmcV_keafJlQymVbQqfbv1g/view?usp=drive_link
# https://drive.google.com/file/d/19k5u0A_cB4XWZdJTALgxEKK-hGD3BvWO/view?usp=drive_link

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

!pip install ultralytics
print("Ultralytics installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
from ultralytics import YOLO


model = YOLO('yolov8n.pt')

print("YOLOv8 model loaded successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 137MB/s]

YOLOv8 model loaded successfully!


In [8]:
import cv2
import os
from google.colab.patches import cv2_imshow


video_path = '/volleyball_match.mp4'


output_video_path = 'outputfinal_volleyball_tracking.mp4'

cap = cv2.VideoCapture(video_path)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))


fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

ball_trajectory = []

# Confidence threshold for detections
conf_threshold = 0.5 # Only show detections with confidence > 50%

print(f"Processing video: {video_path}")
print(f"Total frames: {total_frames}")

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    results = model(frame, verbose=False) # Predict objects in the frame

    # Process results (only one result object per image)
    for r in results:
        boxes = r.boxes  # Boxes object for bounding box outputs
        for box in boxes:
            cls = int(box.cls[0]) # Class ID
            conf = float(box.conf[0]) # Confidence score


            if cls == 32 and conf > conf_threshold: # Class ID 32 is 'sports ball' in COCO
                x1, y1, x2, y2 = map(int, box.xyxy[0]) # Get bounding box coordinates

                # Drawing bounding box on the frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2) # Green box

                # Getting center of the ball and adding to trajectory
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                ball_trajectory.append((center_x, center_y))

                # Drawing a small circle at the current ball location
                cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1) # Red dot

                # Putting label
                label = f"Ball: {conf:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Drawing the trajectory so far
    for i in range(1, len(ball_trajectory)):
        cv2.line(frame, ball_trajectory[i-1], ball_trajectory[i], (255, 0, 0), 2) # Blue line

    # Write the frame to the output video
    out.write(frame)



# Releases everything when the job is finished
cap.release()
out.release()

print(f"\nProcessing complete! Output video saved to {output_video_path}")



Processing video: /volleyball_match.mp4
Total frames: 3437

Processing complete! Output video saved to outputfinal_volleyball_tracking.mp4


In [9]:
import cv2
import numpy as np
import os
from ultralytics import YOLO
from collections import defaultdict


video_path = '/volleyball_match.mp4'
output_video_path = 'output_volleyball_assignment_yellow_favored.mp4'

# Fixed rectangle (ROI) where players are expected(to eliminate other disturbances)
roi_x_min, roi_y_min, roi_x_max, roi_y_max = 200, 200, 1100, 700
conf_threshold_person = 0.1 # Very low confidence threshold for person detection

# --- TEAM CLASSIFICATION RULE(ROI+NEtline divide+Assymetric bufferZone+Team retention) ---
# ADJUSTED: Shifted net line downwards (towards Orange) to give more space for Yellow players
 #(coz the video is not shot from exactly top view therefore requiring an adjustment of a new virtual netline).
TEAM_DIVIDE_Y = 430 # Net line set at Y=430 (Increased from the actual midline - moving it lower on screen)

# Asymmetrical Buffer zone for sticky team assignment
# Players within these pixels of TEAM_DIVIDE_Y will maintain their last known team.
# ADJUSTED: Reduced for the yellow side, bringing the clear yellow zone closer to the net.
BUFFER_ABOVE_LINE = 5 # Pixels extending ABOVE TEAM_DIVIDE_Y for the sticky zone
# Maintained/increased slightly to give Yellow more sticky ownership below the net.
BUFFER_BELOW_LINE = 50 # Pixels extending BELOW TEAM_DIVIDE_Y for the sticky zone

NET_LINE_THICKNESS = 2 # Drawing thickness for the main team division line

TEAM_UPPER_SIDE_NAME = 'YELLOW'
TEAM_LOWER_SIDE_NAME = 'ORANGE'

model = YOLO('yolov8n.pt')


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video file at {video_path}")
    raise FileNotFoundError(f"Video file not found or cannot be opened at: {video_path}")

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# We need a dictionary to store the *last known* team for each tracked player ID
player_last_known_team = defaultdict(lambda: None) # Initialize with None

print("\n Starting video processing")
print(f"Team division line set at Y = {TEAM_DIVIDE_Y} (shifted towards Orange).")
print(f"Asymmetrical sticky buffer zone: {BUFFER_ABOVE_LINE} pixels above, {BUFFER_BELOW_LINE} pixels below.")
print("This setup favors Yellow team's classification space near the net.")
print("Players within the buffer zone will maintain their previous team assignment.")

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Drawing the hard-coded team division line on the frame
    cv2.line(frame, (0, int(TEAM_DIVIDE_Y)), (width, int(TEAM_DIVIDE_Y)), (0, 255, 0), NET_LINE_THICKNESS) # Main green line

    # Drawing the asymmetrical buffer zone lines (darker green)
    # Upper buffer line (closer to net now for Yellow side)
    cv2.line(frame, (0, int(TEAM_DIVIDE_Y - BUFFER_ABOVE_LINE)), (width, int(TEAM_DIVIDE_Y - BUFFER_ABOVE_LINE)), (0, 100, 0), 1)
    # Lower buffer line (more generous for Yellow side)
    cv2.line(frame, (0, int(TEAM_DIVIDE_Y + BUFFER_BELOW_LINE)), (width, int(TEAM_DIVIDE_Y + BUFFER_BELOW_LINE)), (0, 100, 0), 1)

    # Drawing the ROI for visual reference
    cv2.rectangle(frame, (roi_x_min, roi_y_min), (roi_x_max, roi_y_max), (255, 255, 0), 2)


    results = model.track(frame, persist=True, tracker="bytetrack.yaml", verbose=False)

    team_yellow_count = 0
    team_orange_count = 0

    if results[0].boxes.id is not None: # Check if any objects were detected with IDs
        boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)
        ids = results[0].boxes.id.cpu().numpy().astype(int)
        classes = results[0].boxes.cls.cpu().numpy().astype(int)
        confs = results[0].boxes.conf.cpu().numpy()

        for box, p_id, cls, conf in zip(boxes, ids, classes, confs):
            x1, y1, x2, y2 = box

            # --- PERSON DETECTION ---
            if cls == 0 and conf > conf_threshold_person:
                bbox_center_x = (x1 + x2) // 2
                bbox_center_y = (y1 + y2) // 2

                # Step 1: Checking if 'person' is within the defined ROI
                if not (roi_x_min <= bbox_center_x <= roi_x_max and
                        roi_y_min <= bbox_center_y <= roi_y_max):
                    continue # Not in ROI, skip this person

                # Step 2: Player validation (OCR is currently disabled for reliability)
                is_player = True

                if is_player: # Only proceed if the 'person' is considered a 'player'
                    # Step 3: Assign Team using Asymmetrical Sticky Buffer Approach
                    player_foot_y = y2 # Use bottom of bbox as 'foot' position

                    current_team = ""

                    # Clear Yellow zone: player's foot is above the *upper* buffer line
                    if player_foot_y < (TEAM_DIVIDE_Y - BUFFER_ABOVE_LINE):
                        current_team = TEAM_UPPER_SIDE_NAME # Clearly Yellow
                    # Clear Orange zone: player's foot is below the *lower* buffer line
                    elif player_foot_y > (TEAM_DIVIDE_Y + BUFFER_BELOW_LINE):
                        current_team = TEAM_LOWER_SIDE_NAME # Clearly Orange
                    # If within the asymmetrical buffer zone, use sticky assignment or initial based on position
                    else:
                        if player_last_known_team[p_id] is not None:
                            current_team = player_last_known_team[p_id] # Stick to last known team
                        else:
                            # First time seeing this player in the buffer, assign based on which side of the exact line they are.
                            # This still uses the central TEAM_DIVIDE_Y for initial assignment in the ambiguous zone.
                            if player_foot_y <= TEAM_DIVIDE_Y:
                                current_team = TEAM_UPPER_SIDE_NAME
                            else:
                                current_team = TEAM_LOWER_SIDE_NAME

                    # Update the last known team for this player for stickiness in future frames
                    player_last_known_team[p_id] = current_team

                    # Step 4: Update team counts and draw visuals on the frame
                    display_color_bbox = (128, 128, 128) # Default to grey

                    if current_team == TEAM_UPPER_SIDE_NAME:
                        team_yellow_count += 1
                        display_color_bbox = (0, 255, 255) # Bright Yellow for display
                    elif current_team == TEAM_LOWER_SIDE_NAME:
                        team_orange_count += 1
                        display_color_bbox = (0, 165, 255) # Orange for display

                    cv2.rectangle(frame, (x1, y1), (x2, y2), display_color_bbox, 2)
                    label = f"P{p_id} ({current_team})"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, display_color_bbox, 2)

    # --- Display overall Team Counts on the top-left of the frame ---
    cv2.putText(frame, f'Team {TEAM_UPPER_SIDE_NAME}: {team_yellow_count}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Team {TEAM_LOWER_SIDE_NAME}: {team_orange_count}', (10, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 165, 255), 2, cv2.LINE_AA)

    # Write the processed frame to the output video file
    out.write(frame)

cap.release()
out.release()

print(f"\nProcessing complete! Output video saved to {output_video_path}")


 Starting video processing
Team division line set at Y = 430 (shifted towards Orange).
Asymmetrical sticky buffer zone: 5 pixels above, 50 pixels below.
This setup favors Yellow team's classification space near the net.
Players within the buffer zone will maintain their previous team assignment.
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


Processing complete! Output video saved to output_volleyball_assignment_yellow_favored.mp4
